In [304]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

import chromadb 
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
from llama_index.core import load_index_from_storage
from llama_index.core import Document
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import FunctionTool, QueryEngineTool
from llama_index.core.agent import ReActAgent

from sklearn.metrics.pairwise import cosine_similarity

from dotenv import load_dotenv, find_dotenv 
import os 
from llama_index.llms.groq import Groq 

import gradio as gr
import pandas as pd 
from pathlib import Path

In [305]:
def consultar_notas(params: dict):
    """
    params: dict com chaves opcionais:
        turma, ano_letivo, matriculas (lista), alunos (lista),
        disciplina, bimestre (string "1", "2", "3", "4"),
        nota_maxima (float)
    """
    caminho_csv = "Notas_alunos.csv"
    df = pd.read_csv(caminho_csv)

    turma = params.get("turma")
    ano_letivo = params.get("ano_letivo")
    matriculas = params.get("matriculas")
    alunos = params.get("alunos")
    disciplina = params.get("disciplina")
    bimestre = params.get("bimestre")
    nota_maxima = params.get("nota_maxima", 7.0)

    df_filtrado = df

    if turma:
        df_filtrado = df_filtrado[df_filtrado['turma'].str.lower() == turma.lower()]
    if ano_letivo:
        df_filtrado = df_filtrado[df_filtrado['ano_letivo'] == int(ano_letivo)]
    if matriculas:
        df_filtrado = df_filtrado[df_filtrado['matricula'].isin(matriculas)]
    if alunos:
        df_filtrado = df_filtrado[
            df_filtrado['aluno'].apply(lambda x: any(a.lower() in x.lower() for a in alunos))
        ]
    if disciplina:
        df_filtrado = df_filtrado[df_filtrado['disciplina'].str.lower() == disciplina.lower()]
    if bimestre:
        coluna_nota = f'nota_{bimestre}bim'
        if coluna_nota not in df_filtrado.columns:
            return "Bimestre inválido."
        df_filtrado = df_filtrado[df_filtrado[coluna_nota] < float(nota_maxima)]

    if df_filtrado.empty:
        return "Nenhum aluno encontrado com os filtros e nota abaixo do limite."

    respostas = []
    for _, row in df_filtrado.iterrows():
        nota = row.get(f'nota_{bimestre}bim') if bimestre else "N/A"
        respostas.append(
            f"{row['aluno']} (Matrícula: {row['matricula']}) - Nota: {nota} no {bimestre}º bimestre - Turma: {row['turma']}"
        )

    return "\n".join(respostas)

In [306]:
df = pd.read_csv("aulas_todas_materias.csv")
documents = []

for _, row in df.iterrows():
    content = f"Aula: {row['data']}\n\n{row['texto']}" 
    metadata = {
        'data': row['data'],
        'ano': row['ano'],
        'bimestre': f"{row['bimestre']}° bimestre",
        'materia': row['materia'].lower()
    }
    doc = Document(text=content, metadata=metadata)
    documents.append(doc)

documents



[Document(id_='3073e999-0c7c-47a1-871e-0093cdc6fbb6', embedding=None, metadata={'data': '2 ABR', 'ano': 2024, 'bimestre': '2° bimestre', 'materia': 'matematica'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text='Aula: 2 ABR\n\n2 ABR\nOBJETIVOS DE APRENDIZAGEM \nDO DC-GOEM - (GO-\nEMMAT501A) Compreender o \nconceito de função polinomial do 1º\ngrau, identificando a relação entre \nduas variáveis apresentadas em\ntextos de origem socioeconômicas \ne/ou de natureza técnico ou\ncientífica, entre outros para resolver \nsituações problemas do cotidiano.\nOBJETOS DE CONHECIMENTO\nDO DC-GOEM - Funções\npolinomiais do 1º\ngrau (função afim,\nfunção linear,\nfunção constante,\nfunção identidade)\nAplicação de Atividades via \nrecursos digitais\nRealização de Atividades via recursos \ndigitais', path=None, url=None, mimetype=None), image_resou

In [307]:
print(documents[0].get_metadata_str())

data: 2 ABR
ano: 2024
bimestre: 2° bimestre
materia: matematica


### Documentos em aprtes menores

In [308]:
node_parser = SentenceSplitter(chunk_size=1000)
nodes = node_parser.get_nodes_from_documents(documents, show_progress=True)

Parsing nodes: 100%|██████████| 295/295 [00:00<00:00, 2652.85it/s]


In [309]:
nodes

[TextNode(id_='60e4fde3-2b30-47a6-b397-7f41b5dc5115', embedding=None, metadata={'data': '2 ABR', 'ano': 2024, 'bimestre': '2° bimestre', 'materia': 'matematica'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='3073e999-0c7c-47a1-871e-0093cdc6fbb6', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'data': '2 ABR', 'ano': 2024, 'bimestre': '2° bimestre', 'materia': 'matematica'}, hash='6544dcb058080a680e9603c313c63688249e87f463da48cddbbc7e1899820825')}, metadata_template='{key}: {value}', metadata_separator='\n', text='Aula: 2 ABR\n\n2 ABR\nOBJETIVOS DE APRENDIZAGEM \nDO DC-GOEM - (GO-\nEMMAT501A) Compreender o \nconceito de função polinomial do 1º\ngrau, identificando a relação entre \nduas variáveis apresentadas em\ntextos de origem socioeconômicas \ne/ou de natureza técnico ou\ncientífica, entre outros para resolver \nsituações problemas do cotidiano.\nOBJETOS DE CONHECIMENTO\nDO DC-GOEM - Funções\

### Gerando Embeddings

In [310]:
class ChromaEmbeddingWrapper:
    def __init__(self, model_name):
        self.model = HuggingFaceEmbedding(model_name=model_name)
        self.name = model_name
    
    def __call__(self, texts):
        return self.model.encode(texts, show_progress_bar=False).tolist()

Thought: I understand that the tool `Planos` does not exist. The correct tool name is `Planos de aula`. I'll try again with the correct tool name.
Action: Planos
Action Input: {'input': 'plano de aula para aluno com nota abaixo de 5 em 1A'}
Observation: Error: No such tool named `Planos`.
> Running step edcedcca-b45f-47c3-985c-5037a4853898. Step input: None


In [ ]:
current_file = Path().resolve()
model_folder = current_file / "all-MiniLM-L6-v2"
str(model_folder)

In [ ]:
chroma_client = chromadb.Client()

In [ ]:
try:
    chroma_collection = chroma_client.get_or_create_collection("doccuments_llm")
except Exception as e:
    print(f"Erro ao carregar ou criar embeddings {e}")

### Salvando Embeddings no DB

In [ ]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
embed_model = HuggingFaceEmbedding(model_name=str(model_folder))

In [ ]:
index = VectorStoreIndex(nodes, storage_context=storage_context, embed_model=embed_model)

### Verificar conteúdos semelhantes entre bimestres

In [ ]:
def get_text_for_bimestre(bimestre, materia, query_engine):
    prompt = f"Conteúdo do {bimestre}º bimestre de {materia}"
    response = query_engine.query(prompt)
    if hasattr(response, "response"):
        return response.response
    elif isinstance(response, str):
        return response
    else:
        return None
    
def get_embedding(text, embed_model):
    return embed_model([text])[0]


def check_revision_needed(bimestre_atual, materia, query_engine, embed_model, threshold=0.7):
    if bimestre_atual <= 1:
        return False
    
    texto_atual = get_text_for_bimestre(bimestre_atual, materia, query_engine)
    texto_anterior = get_text_for_bimestre(bimestre_atual - 1, materia, query_engine)

    if not texto_atual or not texto_anterior:
        return False
    
    emb_atual = get_embedding(texto_atual, embed_model)  # usa a função com embed_documents
    emb_ant = get_embedding(texto_anterior, embed_model)
    
    sim = cosine_similarity([emb_atual], [emb_ant])[0][0]
    print(f"Similaridade entre bimestre {bimestre_atual} e {bimestre_atual - 1} para {materia}: {sim:.4f}")
    
    return sim >= threshold


### Recuperação de informações

In [ ]:
load_dotenv(find_dotenv())

In [ ]:
GROQ_API = os.environ.get("GROQ_API")

In [ ]:
llm = Groq(
    model="llama3-70b-8192",
    api_key=GROQ_API
)

In [ ]:
query_engine = index.as_query_engine(llm=llm, similarity_top_k=6)

#### Testando o modelo

In [ ]:
aluno = "João da Silva"
nota = 4.5
bimestre = 1
materia = "matematica"

def gerar_plano_estudo(aluno, nota, bimestre, materia, query_engine=None):
    if nota >= 8:
        return f"O aluno {aluno} teve nota {nota} no {bimestre}º bimestre. Nenhum plano de estudo é necessário."

    # Filtra os documentos do bimestre e matéria corretos
    docs_filtrados = [
        doc for doc in documents
        if doc.metadata['bimestre'] == f"{bimestre}° bimestre" and doc.metadata['materia'] == materia.lower()
    ]
    if not docs_filtrados:
        return "Nenhum conteúdo encontrado para esse bimestre/matéria."

    # Cria novo index e query_engine só com os docs filtrados
    nodes_filtrados = node_parser.get_nodes_from_documents( )
    index_filtrado = VectorStoreIndex(nodes_filtrados, embed_model=embed_model)
    query_engine_filtrado = index_filtrado.as_query_engine(llm=llm, similarity_top_k=6)

    prompt = f"""
    O aluno {aluno} obteve nota {nota} no {bimestre}º bimestre na matéria de {materia}.

    Você é um tutor inteligente. Gere um plano de estudo **EXCLUSIVAMENTE** com base no conteúdo de {materia.upper()} do **{bimestre}º bimestre**.

    Ignore completamente conteúdos de outros bimestres, mesmo que relacionados.
    Use **apenas** o que foi ensinado no {bimestre}º bimestre da matéria {materia.upper()}.

    Com base nisso, crie:
    - Conteúdos principais a revisar
    - Dicas de como estudar
    - Sugestões de atividades práticas

    Tudo em português e focado no nível do aluno com nota {nota} e ignora a data que o texto tem foca apenas no bimestre.
    """

    resp = query_engine_filtrado.query(prompt)
    return resp

In [ ]:
gerar_plano_estudo(aluno="pedro", nota=6.5, bimestre=2, materia="Matematica", query_engine=query_engine)

In [ ]:
import gradio as gr

def interface_plano_com_feedback(aluno, nota, bimestre, materia):
    try:
        plano = gerar_plano_estudo(aluno, nota, bimestre, materia, query_engine)
        return plano
    except Exception as e:
        return f"Erro: {str(e)}"

with gr.Blocks() as demo:
    aluno = gr.Textbox(label="Nome do aluno")
    nota = gr.Number(label="Nota do aluno")
    bimestre = gr.Number(label="Bimestre")
    materia = gr.Textbox(label="Matéria")

    btn_gerar = gr.Button("Gerar Plano de Estudo")
    saida_plano = gr.Textbox(label="Plano de Estudo", lines=10)

    btn_gerar.click(interface_plano_com_feedback, inputs=[aluno, nota, bimestre, materia], outputs=saida_plano)

demo.launch(share=True)


In [ ]:
nota_tool = FunctionTool.from_defaults(
    fn=consultar_notas,
    name="ConsultarNotas",
    description="Consulta alunos com nota abaixo de um limite filtrando por turma, disciplina, bimestre, etc. Recebe os filtros como dicionário."
)


In [ ]:
retriever = index.as_retriever(similarity_top_k=6)
retriever_tool = QueryEngineTool.from_defaults(
    #query_engine=retriever,
    query_engine=index.as_query_engine(llm=llm, similarity_top_k=6),
    name="Planos de aula",
    description="Ferramenta que busca planos de aula, conteúdos pedagógicos e atividades relacionados à disciplina, bimestre, turma ou aluno. Use esta ferramenta para qualquer consulta sobre plano de ensino, conteúdo, planejamento, atividades ou reforço escolar.",
)

In [ ]:


chat_engine = ReActAgent.from_tools(
    tools=[nota_tool, retriever_tool],
    llm=llm,
    verbose=True,
    max_iterations=10
)
#index.as_query_engine(llm=llm,)


In [ ]:
def responder(mensagem, chat_history):
    resposta = chat_engine.chat(mensagem).response
    
    if chat_history is None:
        chat_history = []
    
    chat_history.append(("Você", mensagem))
    chat_history.append(("Assistente", resposta))
    
    return chat_history, chat_history

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    entrada = gr.Textbox(placeholder="Digite sua pergunta ou comando...")
    estado = gr.State([])

    entrada.submit(responder, inputs=[entrada, estado], outputs=[chatbot, estado])
    entrada.submit(lambda: "", None, entrada)

demo.launch()

In [ ]:
results = retriever.retrieve("Conteúdo do 4º bimestre de matemática")
for doc in results:
    print(doc.get_text())